In [ ]:
!pip install langchain-google-genai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 792.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 921.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


In [ ]:
from google.colab import userdata
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
model = GoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=GEMINI_API_KEY)
result=model.invoke("What is the capital of India?")
print(result)

The capital of India is **New Delhi**.


In [ ]:
while True:
  user_input=input("You: ")
  if user_input == 'exit':
    break
  result = model.invoke(user_input)
  print("AI: ",result)

You: what is the capital of india?
AI:  The capital of India is **New Delhi**.
You: Who is the prime minister of india?
AI:  The current Prime Minister of India is **Narendra Modi**.
You: exit


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage

model=GoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=GEMINI_API_KEY)

chat_history=[
    SystemMessage(content="You are a helpful AI assistant")
]

while True:
  user_input=input("You:")
  chat_history.append(HumanMessage(content=user_input))
  if user_input == 'exit':
    break
  result=model.invoke(chat_history)
  chat_history.append(AIMessage(content=result))
  print("Proton:",result)

print(chat_history)

You:hi
Proton: Hello! How can I help you today?
You:who is the prime minister pf india
Proton: The current Prime Minister of India is **Narendra Modi**.
You:when is his birthday
Proton: His birthday is on **September 17th**.
You:exit
[SystemMessage(content='You are a helpful AI assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='who is the prime minister pf india', additional_kwargs={}, response_metadata={}), AIMessage(content='The current Prime Minister of India is **Narendra Modi**.', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='when is his birthday', additional_kwargs={}, response_metadata={}), AIMessage(content='His birthday is on **September 17th**.', additional_kwargs={}, response_metadata

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
chat_template=ChatPromptTemplate([
    ('system','You are a helpful {domain} expert'),
    ('human','Explain in short and simple terms, what is {topic}')
])

prompt = chat_template.invoke({'domain':'science','topic':'Photosynthesis'})
print(prompt)

messages=[SystemMessage(content='You are a helpful science expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain in short and simple terms, what is Photosynthesis', additional_kwargs={}, response_metadata={})]


In [ ]:
print(model.invoke(prompt))

Photosynthesis is how **plants make their own food!**

They use:
*   **Sunlight** for energy
*   **Water**
*   **Carbon Dioxide** (from the air)

To create:
*   **Sugar** (their food)
*   **Oxygen** (which we breathe!)


In [ ]:
domain = input("Enter the domain(eg, cricket, science, etc.):")

chat_template=ChatPromptTemplate([
    ('system','You are a helpful {domain} expert who explains things in short and simple terms ')
])
prompt_value = chat_template.invoke({'domain':domain})
messages = prompt_value.to_messages()
chat_history = [messages[0]]

while True:
  user_input = input("You: ")
  if user_input == 'exit':
    break

  chat_history.append(HumanMessage(content=user_input))
  response = model.invoke(chat_history)
  chat_history.append(AIMessage(content=response))
  print("Proton: ",response)
print("\nFinal Chat History:")
for msg in chat_history:
  role="You" if isinstance(msg,HumanMessage) else "Proton" if isinstance(msg,AIMessage) else "System"
  print(f"{role}: {msg}")

Enter the domain(eg, cricket, science, etc.):cricket
You: what is the chemical formula of common salt
Proton:  It's **NaCl**!
You: exit

Final Chat History:
System: content='You are a helpful cricket expert who explains things in short and simple terms ' additional_kwargs={} response_metadata={}
You: content='what is the chemical formula of common salt' additional_kwargs={} response_metadata={}
Proton: content="It's **NaCl**!" additional_kwargs={} response_metadata={} tool_calls=[] invalid_tool_calls=[]


In [ ]:
import gradio as gr
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import  ChatPromptTemplate

model = GoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=GEMINI_API_KEY)

def start_chat(domain):
  chat_template=ChatPromptTemplate([
      ('system',f'You are a helpful {domain} expert who explains things in short and simple terms')
  ])
  prompt_value = chat_template.invoke({'domain':domain})
  messages = prompt_value.to_messages()

  return messages, gr.update(visible=True)

def chat_fn(user_msg, history, messages):
  if user_msg.strip() == '':
    return history, messages

    messages.append(HumanMessage(content=user_msg))
    response = model.invoke(messages)
    messages.append(AIMessage(content=response))

    history.append('You: ',+user_msg,"AI: "+response)
    return history, messages

  with gr.Blocks() as demo:
    gr.Markdown(" ## Domain Expert Chatbot (Proton)")
    domain_input = gr.Textbox(label = "Enter domain (eg. Cricket, Science, etc.)")
    start_btn = gr.Button("Start Chat")
    chatbot = gr.Chatbot(visible=False)
    user_msg = gr.Textbox(label="Your Message")

    messages_status = gr.State([])
    history_state =gr.State([])

    start_btn.click(
        fn=start_chat,
        inputs=[domain_input],
        outputs=[messages_state, chatbot]
    )

    user_msg.submit(
        fn=chat_fn,
        inputs=[user_msg, chatbot, messages_state],
        outputs=[chatbot, messages_state]
    )
  demo.launch()

In [ ]:
import gradio as gr
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import  ChatPromptTemplate

model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=GEMINI_API_KEY)

def start_chat(domain):
  chat_template=ChatPromptTemplate([
      ('system',f'You are a helpful {domain} expert who explains things in short and simple terms')
  ])
  prompt_value = chat_template.invoke({'domain':domain})
  messages = prompt_value.to_messages()

  return messages, gr.update(visible=True)

def chat_fn(user_msg, history, messages):
  if user_msg.strip() == '':
    return history, messages

    messages.append(HumanMessage(content=user_msg))
    response = model.invoke(messages)
    messages.append(AIMessage(content=response))

    history.append('You: ',+user_msg,"AI: "+response)
    return history, messages

  with gr.Blocks() as demo:
    gr.Markdown(" ## Domain Expert Chatbot (Proton)")
    domain_input = gr.Textbox(label = "Enter domain (eg. Cricket, Science, etc.)")
    start_btn = gr.Button("Start Chat")
    chatbot = gr.Chatbot(visible=False)
    user_msg = gr.Textbox(label="Your Message")

    messages_status = gr.State([])
    history_state =gr.State([])

    start_btn.click(
        fn=start_chat,
        inputs=[domain_input],
        outputs=[messages_state, chatbot]
    )

    user_msg.submit(
        fn=chat_fn,
        inputs=[user_msg, chatbot, messages_state],
        outputs=[chatbot, messages_state]
    )
  demo.launch()